### Run slope for each raster in folder

In [1]:
rst_folder = '/mnt/d/mystuff/fireloc/geodb_api/dem_data/dem888_large/'
expo_folder = '/mnt/d/mystuff/fireloc/geodb_api/dem_data/dem888_aspect_large/'
out_name = 'exp'

In [2]:
import os
import pandas as pd
import re
import multiprocessing as mp
from glass.g.wenv.grs import run_grass
from glass.pys.oss import cpu_cores, lst_ff
from glass.pys.oss import fprop
from glass.ng.pd.split import df_split

In [3]:
def run_aspect(tid, inrsts, outfolder, oname):
    iirsts = inrsts.mdt.tolist()
    
    # Create GRASS GIS location
    loc_name = f'thread_{str(tid)}'
    gbase = run_grass(outfolder, location=loc_name, srs=iirsts[0])
    
    # Start GRASS GIS Session
    import grass.script as grass
    import grass.script.setup as gsetup
    gsetup.init(gbase, outfolder, loc_name, 'PERMANENT')
    
    from glass.g.it.rst   import rst_to_grs, grs_to_rst
    from glass.g.rst.surf import aspect
    from glass.g.wenv.grs import rst_to_region
    
    for rst in iirsts:
        # Import data
        mdt = rst_to_grs(rst, fprop(rst, 'fn'))
    
        # Set region
        rst_to_region(mdt)
        
        # Get ID in name
        mdt_id = re.search(r'\d+', mdt).group()
    
        # Get aspect
        rst_aspect = aspect(mdt, f"{oname}_{mdt_id}", from_north=True)
    
        # Export
        grs_to_rst(rst_aspect, os.path.join(outfolder, rst_aspect + '.tif'))

In [4]:
# List Rasters

rsts = pd.DataFrame(lst_ff(rst_folder, file_format='.tif'), columns=['mdt'])

# Split rasters by threads
ncpu = cpu_cores()

dfs = df_split(rsts, ncpu)

# Run slope using multiprocessing
thrds = [mp.Process(
    target=run_aspect, name=f'th_{str(i)}',
    args=(i+1, dfs[i], expo_folder, out_name)
) for i in range(len(dfs))]

for t in thrds:
    t.start()

for t in thrds:
    t.join()